<a href="https://colab.research.google.com/github/csukuangfj/colab/blob/master/sherpa_onnx_c_api_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This colab notebook shows how to use the C API from [sherpa-onnx][sherpa-onnx].

Please refer to
https://k2-fsa.github.io/sherpa/onnx/c-api/index.html
for details

[sherpa-onnx]: https://github.com/k2-fsa/sherpa-onnx

# Install sherpa-onnx

First, let us build `sherpa-onnx` to generate required libraries and header files.

We will build shared libraries in the colab notebook.

Please refer to
https://k2-fsa.github.io/sherpa/onnx/c-api/index.html
for how to build static libraries.

In [1]:
%%shell

# We will install sherpa-onnx into
# /tmp/sherpa-onnx/shared

git clone https://github.com/k2-fsa/sherpa-onnx
cd sherpa-onnx

mkdir build-shared
cd build-shared

cmake \
  -DSHERPA_ONNX_ENABLE_C_API=ON \
  -DCMAKE_BUILD_TYPE=Release \
  -DBUILD_SHARED_LIBS=ON \
  -DCMAKE_INSTALL_PREFIX=/tmp/sherpa-onnx/shared \
  ..

make -j
make install

Cloning into 'sherpa-onnx'...
remote: Enumerating objects: 2619, done.
remote: Counting objects: 100% (1072/1072), done.
remote: Compressing objects: 100% (435/435), done.
remote: Total 2619 (delta 751), reused 775 (delta 626), pack-reused 1547
Receiving objects: 100% (2619/2619), 1.89 MiB | 7.14 MiB/s, done.
Resolving deltas: 100% (1505/1505), done.
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- CMAKE_BUILD_TYPE: Release
-- CMAKE_INSTALL_PREFIX: /tmp/sherpa-onnx/shared
-- BUILD_SHARED_LIBS ON
-- SHERPA_ONNX_ENABLE_PYTHON OFF
-- SHER

In [6]:
%%shell
sudo apt-get install tree
tree /tmp/sherpa-onnx/shared

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 16 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 0s (373 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 120511 files and directories currently install

# pkg-config

`sherpa-onnx` supports [pkg-config](https://en.wikipedia.org/wiki/Pkg-config).

In [2]:
%%shell

# We need to set the environment variable PKG_CONFIG_PATH
# to tell pkg-config where sherpa-onnx locates
export PKG_CONFIG_PATH=/tmp/sherpa-onnx/shared:$PKG_CONFIG_PATH

# Show cflags for gcc
pkg-config --cflags sherpa-onnx

# Show libs for gcc
pkg-config --libs sherpa-onnx


-I/tmp/sherpa-onnx/shared/include
-L/tmp/sherpa-onnx/shared/lib -lsherpa-onnx-c-api -lsherpa-onnx-core -lonnxruntime -lkaldi-native-fbank-core -lcargs -Wl,-rpath,/tmp/sherpa-onnx/shared/lib


# Build C API exmaple

We have provided a file
https://github.com/k2-fsa/sherpa-onnx/blob/master/c-api-examples/decode-file-c-api.c
to demonstrate how to use the C API of `sherpa-onnx`.

In the following, we describe how to build it.

In [3]:
%%shell

# We need to set the environment variable PKG_CONFIG_PATH
# to tell pkg-config where sherpa-onnx locates
export PKG_CONFIG_PATH=/tmp/sherpa-onnx/shared:$PKG_CONFIG_PATH

cd sherpa-onnx/c-api-examples/
gcc -o decode-file-c-api $(pkg-config --cflags sherpa-onnx) ./decode-file-c-api.c $(pkg-config --libs sherpa-onnx)

ls -lh

file decode-file-c-api

ldd decode-file-c-api

./decode-file-c-api --help


total 44K
-rw-r--r-- 1 root root  176 Aug 10 05:16 CMakeLists.txt
-rwxr-xr-x 1 root root  22K Aug 10 05:19 decode-file-c-api
-rw-r--r-- 1 root root 5.9K Aug 10 05:16 decode-file-c-api.c
-rw-r--r-- 1 root root  399 Aug 10 05:16 Makefile
-rwxr-xr-x 1 root root 1.3K Aug 10 05:16 run.sh
decode-file-c-api: ELF 64-bit LSB pie executable, x86-64, version 1 (SYSV), dynamically linked, interpreter /lib64/ld-linux-x86-64.so.2, BuildID[sha1]=0f58ca6805b1ea5e37ad52490fd3d2da27a59d16, for GNU/Linux 3.2.0, not stripped
	linux-vdso.so.1 (0x00007ffefef9d000)
	libsherpa-onnx-c-api.so => /tmp/sherpa-onnx/shared/lib/libsherpa-onnx-c-api.so (0x00007ad984253000)
	libcargs.so => /tmp/sherpa-onnx/shared/lib/libcargs.so (0x00007ad98424e000)
	libc.so.6 => /lib/x86_64-linux-gnu/libc.so.6 (0x00007ad984011000)
	libsherpa-onnx-core.so => /tmp/sherpa-onnx/shared/lib/libsherpa-onnx-core.so (0x00007ad983f4e000)
	libonnxruntime.so.1.15.1 => /tmp/sherpa-onnx/shared/lib/libonnxruntime.so.1.15.1 (0x00007ad982f58000)
	lib

## Test with pre-trained models

The provided `decode-file-c-api.c` supports only streaming transducer models, though C APIs do also support non-streaming models. Please see
https://github.com/k2-fsa/sherpa-onnx/blob/master/sherpa-onnx/c-api/c-api.h
for available C APIs.

In the following, we describe how to use the generated exectuable `decode-file-c-api` to decode files.

First, let us follow the above output to download pre-trained models.

We use
https://k2-fsa.github.io/sherpa/onnx/pretrained_models/online-transducer/zipformer-transducer-models.html#csukuangfj-sherpa-onnx-streaming-zipformer-en-2023-06-26-english
as an example below.

In [4]:
%%shell

sudo apt-get install git-lfs
cd sherpa-onnx/c-api-examples/

GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/csukuangfj/sherpa-onnx-streaming-zipformer-en-2023-06-26
cd sherpa-onnx-streaming-zipformer-en-2023-06-26
git lfs pull --include "*.onnx"
ls -lh *.onnx
cd ..

echo "-----------test fp32 models---------"
# fp32
./decode-file-c-api \
  --tokens=./sherpa-onnx-streaming-zipformer-en-2023-06-26/tokens.txt \
  --encoder=./sherpa-onnx-streaming-zipformer-en-2023-06-26/encoder-epoch-99-avg-1-chunk-16-left-64.onnx \
  --decoder=./sherpa-onnx-streaming-zipformer-en-2023-06-26/decoder-epoch-99-avg-1-chunk-16-left-64.onnx \
  --joiner=./sherpa-onnx-streaming-zipformer-en-2023-06-26/joiner-epoch-99-avg-1-chunk-16-left-64.onnx \
  ./sherpa-onnx-streaming-zipformer-en-2023-06-26/test_wavs/0.wav

echo "-----------test int8 models---------"

# int8
./decode-file-c-api \
  --tokens=./sherpa-onnx-streaming-zipformer-en-2023-06-26/tokens.txt \
  --encoder=./sherpa-onnx-streaming-zipformer-en-2023-06-26/encoder-epoch-99-avg-1-chunk-16-left-64.int8.onnx \
  --decoder=./sherpa-onnx-streaming-zipformer-en-2023-06-26/decoder-epoch-99-avg-1-chunk-16-left-64.int8.onnx \
  --joiner=./sherpa-onnx-streaming-zipformer-en-2023-06-26/joiner-epoch-99-avg-1-chunk-16-left-64.int8.onnx \
  ./sherpa-onnx-streaming-zipformer-en-2023-06-26/test_wavs/0.wav


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.
Cloning into 'sherpa-onnx-streaming-zipformer-en-2023-06-26'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 27 (delta 2), reused 27 (delta 2), pack-reused 0
Unpacking objects: 100% (27/27), 667.63 KiB | 6.74 MiB/s, done.
-rw-r--r-- 1 root root  1.3M Aug 10 05:19 decoder-epoch-99-avg-1-chunk-16-left-128.int8.onnx
-rw-r--r-- 1 root root  2.0M Aug 10 05:19 decoder-epoch-99-avg-1-chunk-16-left-128.onnx
-rw-r--r-- 1 root root  1.3M Aug 10 05:19 decoder-epoch-99-avg-1-chunk-16-left-64.int8.onnx
-rw-r--r-- 1 root root  2.0M Aug 10 05:19 decoder-epoch-99-avg-1-chunk-16-left-64.onnx
-rw-r--r-- 1 root root   68M Aug 10 05:19 encoder-epoch-99-avg-1-chunk-16-left-128.int8.onnx
-rw